In [1]:
import torch
from model import *

In [14]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

device = torch.device('mps')
checkpoint = torch.load('checkpoint/ckpt_20240409_223951.pth', map_location=device)
test_data = unpickle('cifar_test_nolabels.pkl')

state_dict = checkpoint['net']
state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

In [15]:
model_params = {'num_blocks': [5, 4, 3], 'channels': 64}
model = SiLUResNet(ResidualBlock, **model_params)
model.to(device)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [8]:
from torchvision import transforms

norm = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
results = {'ID': [], 'Labels': []}

images = torch.tensor(test_data[b'data']).float()
images = images.view(-1, 3, 32, 32)
images = images / 255.0

with torch.no_grad():
    images = images.to(device)
    outputs = model(images)
    _, predictions = torch.max(outputs, 1)

predictions_arr = predictions.cpu().numpy()

In [10]:
import csv

def write_csv(results):
    # Open the CSV file in write mode
    with open('results.csv', 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['ID', 'Label'])
        
        # Write the header row
        writer.writeheader()
        
        # Write the data rows
        for ids_, prediction in enumerate(results):
            writer.writerow({'ID': ids_, 'Label': prediction})

write_csv(predictions_arr)